In [108]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

TRAIN_DIR =r'C:\Users\Gabi\Desktop\train'
TEST_DIR = r'C:\Users\Gabi\Desktop\test'
IMG_SIZE = 50
LR = 1e-3



In [87]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': return 0
    elif word_label == 'dog': return 1

In [88]:
def creat_train_data():
    training_data=[]
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy',training_data)
    return training_data

In [40]:
train_data = creat_train_data()

100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [00:36<00:00, 688.14it/s]


In [93]:
X = np.array([i[0] for i in train_data]).reshape(25000,2500)
Y = [i[1] for i in train_data]

In [118]:
X_train, X_test, y_train, y_test = train_test_split(
     X, Y, test_size=0.2, random_state=42)

In [ ]:
pca = PCA()
X2D = pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

In [ ]:
d

In [ ]:
plt.plot(cumsum)

In [ ]:
from sklearn.decomposition import IncrementalPCA
n_batches = 40
inc_pca = IncrementalPCA(n_components=187)
for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)

X_train = inc_pca.transform(X_train)


In [91]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)

ValueError: Found array with dim 4. StandardScaler expected <= 2.

In [ ]:
X_test = inc_pca.transform(X_test)
X_test = sc.fit_transform(X_test)

In [94]:
print(X)

[[255 255 255 ... 255 255 255]
 [ 10  11  22 ...  80  76  69]
 [126 101 110 ... 202 220 213]
 ...
 [155 155 144 ...  50  35 123]
 [189 197 193 ... 151 152 153]
 [ 53  95 100 ...   8   9   6]]


In [95]:
print(Y)

[array(0), array(1), array(0), array(0), array(1), array(0), array(0), array(0), array(0), array(0), array(0), array(1), array(1), array(1), array(0), array(0), array(0), array(1), array(0), array(0), array(0), array(1), array(1), array(0), array(1), array(0), array(0), array(1), array(1), array(0), array(1), array(0), array(1), array(0), array(1), array(0), array(0), array(0), array(1), array(1), array(0), array(1), array(0), array(0), array(1), array(1), array(1), array(1), array(1), array(1), array(1), array(0), array(0), array(0), array(0), array(1), array(0), array(1), array(0), array(0), array(1), array(1), array(0), array(1), array(0), array(0), array(0), array(1), array(1), array(1), array(1), array(1), array(1), array(0), array(1), array(1), array(0), array(0), array(1), array(0), array(0), array(0), array(1), array(0), array(1), array(0), array(1), array(1), array(0), array(1), array(1), array(0), array(0), array(1), array(1), array(1), array(1), array(0), array(1), array(0),

In [ ]:
xgb_reg = xgb.XGBClassifier(eval_metric = 'mlogloss')
xgb_reg.fit(X_train, y_train)

In [132]:
xgb_reg.score(X_test, y_test)

0.653

In [133]:
err_rate=[] 
for i in range(1,5):
    k=KNeighborsClassifier(n_neighbors=i)
    k.fit(X_train,y_train)
    bestK=k.predict(X_test)
    err_rate.append(np.mean(bestK != y_test))

plt.plot(range(1,5),err_rate)
plt.xlabel('Best value for K')
plt.ylabel('Accuracy')

KeyboardInterrupt: 

In [ ]:
err_rate=[] 
for i in range(1,100):
    k=KNeighborsClassifier(n_neighbors=i)
    k.fit(X_train,y_train)
    bestK=k.predict(X_test)
    err_rate.append(np.mean(bestK != y_test))

In [ ]:
plt.plot(range(1,100),err_rate)
plt.xlabel('Best value for K')
plt.ylabel('Accuracy')

In [135]:
from sklearn.ensemble import BaggingClassifier
knn = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=27),
                      n_estimators=10, random_state=0).fit(X_train, y_train)

In [ ]:
ranfor = BaggingClassifier(base_estimator=RandomForestClassifier(),
                      n_estimators=10, random_state=0).fit(X_train, y_train)

In [136]:
y_pred = knn.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5802


In [124]:
from sklearn.ensemble import AdaBoostClassifier
ad =AdaBoostClassifier(n_estimators=50, learning_rate=1.0, algorithm='SAMME')

In [125]:
ad.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME')

In [126]:
ad.predict(X_test)
ad.score(X_test,y_test)

0.589

In [139]:
clf77= BaggingClassifier(n_estimators=10, random_state=0).fit(X_train, y_train)

In [137]:
from sklearn.metrics import accuracy_score

In [151]:
voting_clf = VotingClassifier(
 estimators=[('xg', xgb_reg ), ('cf', clf77),('ad', ad),('rn',ranfor)],
 voting='soft')
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)
print(accuracy_score(y_pred, y_test))

C:\Users\Gabi\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.6554


In [141]:
estimators1 = [('xg', xgb_reg ), ('cf', clf77)]

In [145]:
from sklearn.ensemble import StackingClassifier

In [152]:
lala = StackingClassifier(
   estimators=estimators, final_estimator=knn)

In [153]:
lala.fit(X_train, y_train).score(X_test, y_test)

C:\Users\Gabi\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Gabi\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.6428